# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13702

In [5]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [6]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [7]:
sents_tokenize = [tokenizer.tokenize(sent) for sent in sents]
sents_tokenize[4:6]

[['и',
  'каждый',
  'раз',
  'молодой',
  'человек',
  'проходя',
  'мимо',
  'чувствовал',
  'какое',
  'то',
  'болезненное',
  'и',
  'трусливое',
  'ощущение',
  'которого',
  'стыдился',
  'и',
  'от',
  'которого',
  'морщился'],
 ['он',
  'был',
  'должен',
  'кругом',
  'хозяйке',
  'и',
  'боялся',
  'с',
  'нею',
  'встретиться']]

In [8]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [9]:
len(words) # всего слов

173403

In [10]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [11]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [12]:
sents_tokenize = [[word for word in text_cur if word not in stopwords_ru] for text_cur in sents_tokenize]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [14]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

Обработаем все слова из датасета. 

In [15]:
sents_tokenize = [[morph.normal_forms(word)[0] for word in text_cur] for text_cur in sents_tokenize]

In [17]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [18]:
len(words) # всего слов

93069

In [19]:
len(set(words)) # уникальных слов

11153

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [20]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1481),
 ('весь', 1405),
 ('один', 659),
 ('раскольник', 567),
 ('свой', 549),
 ('сказать', 544),
 ('человек', 501),
 ('говорить', 498),
 ('пётр', 492),
 ('знать', 478),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('самый', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('дело', 363)]

In [21]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4793

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [22]:
%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=4)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

CPU times: user 17.2 s, sys: 272 ms, total: 17.5 s
Wall time: 8.94 s


In [23]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [24]:
len(model.wv.vocab)

4793

In [25]:
'старуха' in model.wv.vocab

True

## 3. Свойства модели

In [26]:
# вектор слова
model.wv['старуха'][:10]

array([-0.1891717 , -1.0078331 ,  0.02465007,  1.9336507 ,  0.9836627 ,
       -0.5515374 , -2.1063323 , -1.1825562 , -2.1625562 , -0.8773531 ],
      dtype=float32)

In [27]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [28]:
# похожести слов 
model.wv.similarity('тварь', 'право')

0.13398334

In [29]:
model.wv.similarity('старуха', 'топор')

0.42833737

In [30]:
model.wv.similarity('тварь', 'тварь')

1.0

In [31]:
# самые похожие
model.wv.most_similar('старуха')

[('лизавета', 0.5569199323654175),
 ('наклепать', 0.5483915209770203),
 ('близко', 0.4810512661933899),
 ('чиновница', 0.4743647575378418),
 ('топор', 0.42833733558654785),
 ('грабёж', 0.4256175458431244),
 ('красильщик', 0.4135328531265259),
 ('взмахнуть', 0.40772098302841187),
 ('маляр', 0.3881557881832123),
 ('раздумье', 0.3795427083969116)]

In [32]:
model.wv.most_similar('лестница')

[('ступенька', 0.5321828722953796),
 ('передний', 0.5138054490089417),
 ('всходить', 0.49897563457489014),
 ('смутно', 0.46502870321273804),
 ('ниже', 0.46335119009017944),
 ('спьяну', 0.4606303572654724),
 ('узенький', 0.45165687799453735),
 ('подниматься', 0.43760809302330017),
 ('прихожая', 0.436430424451828),
 ('площадка', 0.4237166941165924)]

In [33]:
# арифметика
model.most_similar(positive=['раскольников'], negative=['тварь'])[:10]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('короткий', 0.30038806796073914),
 ('заглянуть', 0.29947230219841003),
 ('заглядывать', 0.29084068536758423),
 ('кругом', 0.2831892669200897),
 ('разъяснить', 0.27985256910324097),
 ('остановиться', 0.2769550681114197),
 ('видеться', 0.2744836211204529),
 ('увидеть', 0.27435925602912903),
 ('хорошенько', 0.27119874954223633),
 ('бесцеремонно', 0.2687910795211792)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [34]:
model_path = "./our_w2v.model"
model.save(model_path)

In [68]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [69]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [70]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]


In [71]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [72]:
len(sents_tokenize2)

254

Дополняем модель.

In [74]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили
our_model.train(sents_tokenize2, total_examples=our_model.corpus_count, epochs=100)

(178206, 269000)

In [81]:
'ядро' in model.wv.vocab

False

In [82]:
'ядро' in our_model.wv.vocab

True

In [83]:
our_model.wv.most_similar('ядро')

[('скатерть', 0.6989225149154663),
 ('изумруд', 0.6974911093711853),
 ('золотой', 0.6087195873260498),
 ('грызть', 0.5864646434783936),
 ('камень', 0.5648640990257263),
 ('чистый', 0.5592674016952515),
 ('разбиться', 0.5529054403305054),
 ('пристань', 0.5513269901275635),
 ('научить', 0.5490145087242126),
 ('шкатулка', 0.5452994108200073)]

Пример со старым словом.

In [88]:
our_model.wv.most_similar('сын')

[('обеспечить', 0.508002519607544),
 ('наущение', 0.49238407611846924),
 ('секрет', 0.49059611558914185),
 ('подвиг', 0.4897482097148895),
 ('благословить', 0.487861692905426),
 ('окончить', 0.4802014231681824),
 ('губерния', 0.4793137311935425),
 ('испытание', 0.4776143431663513),
 ('литературный', 0.47760239243507385),
 ('жених', 0.47474443912506104)]

In [89]:
model.wv.most_similar('сын')

[('наущение', 0.5248565673828125),
 ('секрет', 0.5242533087730408),
 ('подвиг', 0.498577356338501),
 ('обеспечить', 0.491332471370697),
 ('интерес', 0.48974451422691345),
 ('приём', 0.4830693304538727),
 ('сестрица', 0.4647851884365082),
 ('положение', 0.46402543783187866),
 ('беспредельный', 0.46239274740219116),
 ('выздоровление', 0.46238258481025696)]